In [20]:
import numpy as np
import pickle as pkl
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from collections import Counter
import os
import glob
import csv
import re
import cv2
from matplotlib.pyplot import imshow
%matplotlib inline

videos_loc = "/Volumes/Transcend/Tesi/Video_Berlino/Videos"

First of all let's gather the labels information from the csv files.
Entries are structured as:
001-subject01-screw_driver-normal_surface-impaired_vision-hold-1,atu 0.567-1.934,c2tgn 1.934-2.584
We need to:
1. remove the leading zeros from the trial name to match the folder names
2. drop the information about the seconds
3. extract useful information from the trialID, such as 'hot' or 'impaired'

The final result should look something like 1-subject01-screw_driver-normal_surface-impaired_vision-hold-1 -> ['atu','934,c2tgn']

In [2]:
# drop seconds from labels, they come after a space
seconds_pattern = re.compile('(.*) (.*)')

def extract_label(string):
    # Drop the information about seconds from labels
    # e.g. "abg 0.484-1.134" -> "abg"
    match = re.match(seconds_pattern, string)
    if match is None:
        return None
    else:
        return match.groups()[0]

In [3]:
# e.g. 217-subject15-credit_card-hot_surface-impaired_vision-hold-1
# means that trial 217 of subject 15 was to grab a credit card, 
# avoiding touching the table while wearing frosted goggles

num = '(?P<num>[0-9]+)'
subj = '(?P<subj>[a-z0-9]+)'
obj = '(?P<obj>[a-z_]+)'
surface = '(?P<surface>[a-z_]+)'
vision = '(?P<vision>[a-z_]+)'
hold = '(?P<hold>(hold|use))'
metadata_pattern = re.compile(
    num + '\-' + subj + '\-' + obj + '\-' + surface + '\-' + vision + '\-' + hold + '\-[12]')

valid_labels = ["a", "f", "c", "e", "m", "x", "o", "u", "g", "r", "l",
                "d", "n", "t", "s", "b", "p", "q", "w", "?", "2", "3", "4", "5"]
# generate pattern to check labels
labels_pattern = re.compile('[' + '|'.join(valid_labels) + ']+')

def is_impaired(value):
    return value == 'impaired_vision'

def is_hot(value):
    return value == 'hot_surface'

def extract_metadata(metadata):
    match = metadata_pattern.match(metadata)
    if match is not None:
        return match.groupdict()
    else:
        print('ERROR: {}'.format(metadata))

Is also useful to know if the test subject failed mid trial and had to restart, when that happens the labels contain one or more 'x'

In [4]:
def failed(labels):
     return any('x' in label for label in labels) 

We can now proceed reading the csv files and parsing them with the helper functions and regexes we have declared

In [5]:
labels = [] # we are going to store here the labels
labels_folder = 'labels1' # labels created by a professional 
for file in glob.glob(os.path.join(videos_loc, 'labels1', '*.csv')):
    with open(file, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            # remove leading zeros so names match folders
            name = row[0].lstrip('0')
            # use filter to remove empty strings
            sequence = list(filter(None, map(extract_label, row[1:])))
            if(sequence != []):
                #labels_dict[name] = sequence
                metadata = extract_metadata(name)
                hot = is_hot(metadata['surface'])
                impaired = is_impaired(metadata['vision'])
                obj = metadata['obj']
                subj = metadata['subj']
                fail = failed(sequence)
                newID = subj+'/'+metadata['num']+'-'+obj
                assert len(sequence) > 0
                # check labels contain only valid values
                for labels_list in sequence:
                    assert labels_pattern.match(labels_list) is not None
                labels.append([name,newID,subj, hot, impaired, obj, fail, sequence])

labels_df = pd.DataFrame(labels, columns=['trial','new_location','subj', 'hot','impaired', 'object', 'failed','labels'])

We now have a dataframe with all the information we need about trials and their conditions.

Next we want to associate each TrialID to the folder where the corresponding videos are located.
We could extract this information from the TrialID but there are missing files so we'll explore each folder to check.

In [6]:
def read_camera_files():
    # Folder structure:
    # subject01 ->
    #       1-Subject01-screw driv... ->
    #                               camera1.mp4
    #                               camera2.mp4
    #                               camera3.mp4
    #                               camera4.mp4
    #       2-Subject01-plate-norm... ->
    #                               camera1.mp4
    #                               camera2.mp4
    #                               camera3.mp4
    #                               camera4.mp4
    #       ...
    #
    # subject02 ->
    #       1-Subject01... ->
    #
    files = set(['camera1.mp4', 'camera2.mp4', 'camera3.mp4', 'camera4.mp4'])
    # Get subdirectories of root dataset folder containing trials data
    subjects = glob.glob(os.path.join(videos_loc, '*subject*/'))
    dataset = []
    # for each subject directory
    for subjects in subjects:
        # get the subdirectories with camera data
        subject_folders = glob.glob(os.path.join(subjects, '*/'))
        dataset.append(subject_folders)
    # now dataset contains all the folders we need to extract frames from
    examples = []
    for subj in dataset:
        for el in subj:
            # get id of each experiment from the last folder name
            clean_name = os.path.basename(os.path.normpath(el))
            # change spaces to _ to match labels
            clean_name = clean_name.replace(' ', '_')
            # ids are lowercase
            clean_name = clean_name.lower()
            if set(os.listdir(el)).issuperset(files): # check if all the 4 videos are present
                examples.append((clean_name,el))  # (TrialID,location)
    return pd.DataFrame(examples, columns=['trial','old_location'])

In [7]:
location_df = read_camera_files()

In [8]:
location_df

,trial,old_location
0,99-subject01-credit_card-normal_surface-unimpa...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
1,95-subject01-chestnut-normal_surface-unimpaire...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
2,88-subject01-matchbox-normal_surface-unimpaire...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
3,82-subject01-cd-normal_surface-unimpaired_visi...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
4,81-subject01-game_card-normal_surface-unimpair...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
5,80-subject01-french_chalk-normal_surface-unimp...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
6,8-subject01-shell-normal_surface-impaired_visi...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
7,77-subject01-tape-normal_surface-unimpaired_vi...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
8,72-subject01-button-normal_surface-unimpaired_...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...
9,71-subject01-rubber_band-normal_surface-unimpa...,/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...


We now join labels and folder information based on the TrialID

In [9]:
assert not any(labels_df.duplicated('trial')) # check that there are no duplicates
assert not any(location_df.duplicated('trial'))
metadata_df = pd.merge(labels_df, location_df, on=['trial','trial'])

One last useful piece of information we are going to need is the one hot encoding of the object names.

In [10]:
# binaryze object classes to perform obj recognition
lb = LabelBinarizer()
metadata_df['obj_classes'] = lb.fit_transform(metadata_df['object']).tolist()

In [11]:
metadata_df

,trial,new_location,subj,hot,impaired,object,failed,labels,old_location,obj_classes
0,1-subject01-screw_driver-normal_surface-impair...,subject01/1-screw_driver,subject01,False,True,screw_driver,False,"[atu, c2tgn]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2-subject01-plate-normal_surface-impaired_visi...,subject01/2-plate,subject01,False,True,plate,False,"[abg, c3bgn]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3-subject01-chestnut-normal_surface-impaired_v...,subject01/3-chestnut,subject01,False,True,chestnut,False,"[atu, c4tgn]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4-subject01-shashlik-normal_surface-impaired_v...,subject01/4-shashlik,subject01,False,True,shashlik,False,"[atu, c3tgr]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5-subject01-matchbox-normal_surface-impaired_v...,subject01/5-matchbox,subject01,False,True,matchbox,False,"[atu, c4tgn]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,6-subject01-cd-normal_surface-impaired_vision-...,subject01/6-cd,subject01,False,True,cd,True,"[atu, c4tgn, fp, x, c4tgn, fq, x, fp, x, fp, m...",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,7-subject01-salt_shaker-normal_surface-impaire...,subject01/7-salt_shaker,subject01,False,True,salt_shaker,False,"[atu, c4tun]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,8-subject01-shell-normal_surface-impaired_visi...,subject01/8-shell,subject01,False,True,shell,False,"[atu, c4tgn]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,9-subject01-match-normal_surface-impaired_visi...,subject01/9-match,subject01,False,True,match,True,"[atu, c3tgr, c2tgn, x, mtgl, eg]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
9,10-subject01-cigarette-normal_surface-impaired...,subject01/10-cigarette,subject01,False,True,cigarette,False,"[atu, c2tgr]",/Volumes/Transcend/Tesi/Video_Berlino/Videos/s...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


We can now proceed to extract the frames from the videos. Each folder contains 4 videos and we are going to extract the first frame of each of them.

In [16]:
frames_loc = '/Volumes/Transcend/Tesi/Video_Berlino/Original'
def frames_extractor(from_folder, to_folder):
    """
    Given a folder containing 4 mp4 files it returns the first frames in
    grayscale as a np.array (4,480,640)
    """
    images = []
    files = ['camera1.mp4', 'camera2.mp4', 'camera3.mp4', 'camera4.mp4']
    # build path of desired file
    videos = [os.path.join(from_folder, x) for x in files]
    # check if files exist
    for video in videos:
        if not os.path.isfile(video):
            return None
    # if all files exist proceed to extract frames
    # first we create the folder
    directory = os.path.join(frames_loc,to_folder)
    if os.path.exists(directory):
        print("Skip "+directory)
        return None
    else:
        os.makedirs(directory)
    # then we extract the frames
    for idx,video in enumerate(videos):
        ret = False
        cap = cv2.VideoCapture(video)
        # make sure the video has been opened
        if not cap.isOpened():
            print("Wait for the header of {}".format(video))
            return None
        # width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        # height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        # fps = cap.get(cv2.CAP_PROP_FPS)
        while(ret == False):
            ret, frame = cap.read()
        cap.release()
        
        # drop the extra channels since the image is grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # write the frames in the new folder
        image = os.path.join(directory,'camera'+str(idx)+'.jpg')
        cv2.imwrite(image, gray)

For each entry in our dataframe we are going to extract the frames and save them to the new location.

In [13]:
for row in metadata_df.iterrows():
    old = row[1]['old_location']
    new = row[1]['new_location']
    print('Extracting: {}'.format(new))
    frames_extractor(old,new)

Extracting: subject01/1-screw_driver
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/1-screw_driver
Extracting: subject01/2-plate
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/2-plate
Extracting: subject01/3-chestnut
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/3-chestnut
Extracting: subject01/4-shashlik
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/4-shashlik
Extracting: subject01/5-matchbox
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/5-matchbox
Extracting: subject01/6-cd
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/6-cd
Extracting: subject01/7-salt_shaker
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/7-salt_shaker
Extracting: subject01/8-shell
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/8-shell
Extracting: subject01/9-match
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/9-match
Extracting: subject01/10-cigarette
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/10-ci

Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/91-shashlik
Extracting: subject01/92-book
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/92-book
Extracting: subject01/93-match
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/93-match
Extracting: subject01/94-screw_driver
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/94-screw_driver
Extracting: subject01/95-chestnut
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/95-chestnut
Extracting: subject01/96-toy
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/96-toy
Extracting: subject01/97-shell
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/97-shell
Extracting: subject01/98-rubber_band
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/98-rubber_band
Extracting: subject01/99-credit_card
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/99-credit_card
Extracting: subject01/100-bowl
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject01/100-bowl
Extracting

Extracting: subject03/78-shell
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/78-shell
Extracting: subject03/79-comb
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/79-comb
Extracting: subject03/80-screw
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/80-screw
Extracting: subject03/81-game_card
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/81-game_card
Extracting: subject03/82-marker
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/82-marker
Extracting: subject03/83-bowl
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/83-bowl
Extracting: subject03/84-chestnut
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/84-chestnut
Extracting: subject03/85-book
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/85-book
Extracting: subject03/86-match
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/86-match
Extracting: subject03/87-glasses
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject03/87-glasses
Extrac

Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/212-coffee_mug
Extracting: subject06/213-key
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/213-key
Extracting: subject06/214-shashlik
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/214-shashlik
Extracting: subject06/215-comb
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/215-comb
Extracting: subject06/216-french_chalk
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/216-french_chalk
Extracting: subject06/217-salt_shaker
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/217-salt_shaker
Extracting: subject06/218-credit_card
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/218-credit_card
Extracting: subject06/219-cigarette
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/219-cigarette
Extracting: subject06/220-bowl
Skip /Volumes/Transcend/Tesi/Video_Berlino/Test/subject06/220-bowl
Extracting: subject06/221-screw_driver
Skip /Volumes/Transcend/Tesi/Video_Berlino/T

Extracting: subject07/97-marker
Extracting: subject07/98-comb
Extracting: subject07/99-match
Extracting: subject07/100-screw
Extracting: subject07/101-marker
Extracting: subject07/102-game_card
Extracting: subject07/103-cd
Extracting: subject07/104-book
Extracting: subject07/105-coffee_mug
Extracting: subject07/106-chestnut
Extracting: subject07/107-glasses
Extracting: subject07/108-matchbox
Extracting: subject07/109-button
Extracting: subject07/110-plate
Extracting: subject07/111-shell
Extracting: subject07/112-toy
Extracting: subject07/113-credit_card
Extracting: subject07/114-salt_shaker
Extracting: subject07/115-shashlik
Extracting: subject07/116-rubber_band
Extracting: subject07/117-screw
Extracting: subject07/118-key
Extracting: subject07/119-tape
Extracting: subject07/120-cigarette
Extracting: subject07/121-comb
Extracting: subject07/122-screw_driver
Extracting: subject07/123-french_chalk
Extracting: subject07/124-match
Extracting: subject07/125-bowl
Extracting: subject07/126-ci

...

Extracting: subject16/60-game_card
Extracting: subject16/61-shashlik
Extracting: subject16/62-glasses
Extracting: subject16/63-chestnut
Extracting: subject16/64-credit_card
Extracting: subject16/65-match
Extracting: subject16/66-bowl
Extracting: subject16/67-comb
Extracting: subject16/68-shell
Extracting: subject16/69-toy
Extracting: subject16/70-cigarette
Extracting: subject16/71-book
Extracting: subject16/72-salt_shaker
Extracting: subject16/73-tape
Extracting: subject16/74-marker
Extracting: subject16/75-cd
Extracting: subject16/76-cigarette
Extracting: subject16/77-game_card
Extracting: subject16/78-comb
Extracting: subject16/79-french_chalk
Extracting: subject16/80-bowl
Extracting: subject16/81-key
Extracting: subject16/82-salt_shaker
Extracting: subject16/83-button
Extracting: subject16/84-credit_card
Extracting: subject16/85-marker
Extracting: subject16/86-shell
Extracting: subject16/87-rubber_band
Extracting: subject16/88-screw
Extracting: subject16/89-screw_driver
Extracting: 

Extracting: subject17/56-chestnut
Extracting: subject17/57-toy
Extracting: subject17/58-rubber_band
Extracting: subject17/59-plate
Extracting: subject17/60-salt_shaker
Extracting: subject17/61-match
Extracting: subject17/62-marker
Extracting: subject17/63-comb
Extracting: subject17/64-book
Extracting: subject17/65-cd
Extracting: subject17/66-shashlik
Extracting: subject17/67-game_card
Extracting: subject17/68-key
Extracting: subject17/69-tape
Extracting: subject17/70-matchbox
Extracting: subject17/71-screw
Extracting: subject17/72-credit_card
Extracting: subject17/73-button
Extracting: subject17/74-french_chalk
Extracting: subject17/75-shell
Extracting: subject17/76-cd
Extracting: subject17/77-french_chalk
Extracting: subject17/78-glasses
Extracting: subject17/79-button
Extracting: subject17/80-chestnut
Extracting: subject17/81-salt_shaker
Extracting: subject17/82-credit_card
Extracting: subject17/83-book
Extracting: subject17/84-screw
Extracting: subject17/85-marker
Extracting: subjec

Extracting: subject18/52-comb
Extracting: subject18/53-key
Extracting: subject18/54-glasses
Extracting: subject18/55-matchbox
Extracting: subject18/56-shashlik
Extracting: subject18/57-french_chalk
Extracting: subject18/58-credit_card
Extracting: subject18/59-toy
Extracting: subject18/60-screw
Extracting: subject18/61-marker
Extracting: subject18/62-plate
Extracting: subject18/63-screw_driver
Extracting: subject18/64-button
Extracting: subject18/65-salt_shaker
Extracting: subject18/66-book
Extracting: subject18/67-coffee_mug
Extracting: subject18/68-game_card
Extracting: subject18/69-chestnut
Extracting: subject18/70-cigarette
Extracting: subject18/71-tape
Extracting: subject18/72-shell
Extracting: subject18/73-rubber_band
Extracting: subject18/74-match
Extracting: subject18/75-bowl
Extracting: subject18/76-match
Extracting: subject18/77-credit_card
Extracting: subject18/78-cd
Extracting: subject18/79-glasses
Extracting: subject18/80-comb
Extracting: subject18/81-cigarette
Extracting: 

Extracting: subject19/47-rubber_band
Extracting: subject19/48-shell
Extracting: subject19/49-matchbox
Extracting: subject19/50-button
Extracting: subject19/51-marker
Extracting: subject19/52-bowl
Extracting: subject19/53-tape
Extracting: subject19/54-rubber_band
Extracting: subject19/55-chestnut
Extracting: subject19/56-toy
Extracting: subject19/57-credit_card
Extracting: subject19/58-comb
Extracting: subject19/59-salt_shaker
Extracting: subject19/60-glasses
Extracting: subject19/61-shell
Extracting: subject19/62-button
Extracting: subject19/63-book
Extracting: subject19/64-shashlik
Extracting: subject19/65-cigarette
Extracting: subject19/66-match
Extracting: subject19/67-screw
Extracting: subject19/68-matchbox
Extracting: subject19/69-french_chalk
Extracting: subject19/70-coffee_mug
Extracting: subject19/71-game_card
Extracting: subject19/72-cd
Extracting: subject19/73-screw_driver
Extracting: subject19/74-plate
Extracting: subject19/75-key
Extracting: subject19/76-book
Extracting: su

Extracting: subject20/83-tape
Extracting: subject20/84-screw
Extracting: subject20/85-cd
Extracting: subject20/86-button
Extracting: subject20/87-comb
Extracting: subject20/88-screw_driver
Extracting: subject20/89-credit_card
Extracting: subject20/90-french_chalk
Extracting: subject20/91-match
Extracting: subject20/92-rubber_band
Extracting: subject20/93-glasses
Extracting: subject20/94-book
Extracting: subject20/95-bowl
Extracting: subject20/96-chestnut
Extracting: subject20/97-salt_shaker
Extracting: subject20/98-plate
Extracting: subject20/99-coffee_mug


Since most pre-trained image recognition models work on square images while our frames are (480,640) we are going to create a separate copy cropping a square in the middle of each frame. This is going to reduce the size needed for storage and is not going to ruin the images since the objects are always placed in the middle of the frame.

In [73]:
def square(img,center=[0,0]):
    (h,w),flipped = img.shape,False
    if(h>w):
        h,w,flipped,img = w,h,True,img.T
    trim = (w - h)//2
    img = img[:,trim:h+trim]
    if flipped:
        return img.T
    else:
        return img

In [79]:
squares_loc = '/Volumes/Transcend/Tesi/Video_Berlino/Squared'
for row in metadata_df.iterrows():
    new = row[1]['new_location']
    old_loc = os.path.join(frames_loc,new)
    new_loc = os.path.join(squares_loc,new)
    print(new_loc)
    if os.path.exists(new_loc):
        print("Skip "+ new_loc)
    else:
        os.makedirs(new_loc)
    for i in range(4):
        frame = os.path.join(old_loc,'camera{}.jpg'.format(i))
        old_pic = cv2.imread(frame,cv2.IMREAD_GRAYSCALE)
        new_pic = square(old_pic)
        name = os.path.join(new_loc,'square{}.jpg'.format(i))
        cv2.imwrite(name,new_pic)

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/1-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/2-plate
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/3-chestnut
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/4-shashlik
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/5-matchbox
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/6-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/7-salt_shaker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/8-shell
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/9-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/10-cigarette
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/11-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/12-rubber_band
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/13-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/14-game_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/15-coffee_

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/124-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/125-glasses
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/126-rubber_band
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/128-chestnut
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/129-coffee_mug
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/130-screw
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/131-salt_shaker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/132-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/133-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/134-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/135-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/136-book
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/137-key
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/138-toy
/Volumes/Transcend/Tesi/Video_Ber

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/246-key
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/247-chestnut
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/248-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/249-marker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject01/250-tape
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/1-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/2-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/3-salt_shaker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/4-screw
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/5-book
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/6-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/7-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/8-key
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/9-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/10-glasse

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/119-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/120-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/121-cigarette
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/122-matchbox
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/123-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/124-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/125-screw
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/126-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/127-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/128-matchbox
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/129-rubber_band
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/130-tape
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/131-coffee_mug
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/132-key
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/133

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/240-chestnut
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/241-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/242-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/243-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/244-plate
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/245-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/246-coffee_mug
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/247-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/248-marker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/249-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject02/250-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject03/1-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject03/2-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject03/3-comb
/Volumes/Transcend/Tesi/Video_Berlino/Squared/su

...

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/108-book
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/109-plate
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/110-game_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/111-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/112-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/113-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/114-glasses
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/115-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/116-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/117-marker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/118-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/119-matchbox
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/120-toy
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/121-rubber_band
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/229-glasses
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/230-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/231-marker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/232-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/233-shell
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/234-key
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/235-toy
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/236-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/237-cigarette
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/238-coffee_mug
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/239-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/240-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/241-tape
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/242-screw
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject04/24

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/102-screw
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/103-credit_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/104-french_chalk
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/105-rubber_band
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/106-game_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/107-salt_shaker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/108-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/109-glasses
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/110-coffee_mug
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/111-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/112-toy
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/113-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/114-bowl
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/115-cigarette
/Volumes/Transcend/Tesi/Video_Berl

/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/223-cd
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/224-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/225-plate
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/226-game_card
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/227-tape
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/228-toy
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/229-chestnut
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/230-matchbox
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/231-button
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/232-screw_driver
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/233-book
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/234-marker
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/235-glasses
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/236-match
/Volumes/Transcend/Tesi/Video_Berlino/Squared/subject06/237-ci

AttributeError: 'NoneType' object has no attribute 'shape'

Now that we have created our dataset with the frames, all that's left is to save the dataframe containing the metadata in a compressed pickle, so that we can load it in memory easily when we are going to train our model.

In [81]:
# Save
metadata_df.to_pickle(os.path.join(squares_loc,'metadata.pkl.compress'), compression='gzip')